In [1]:
# 库文件
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd

from matplotlib.font_manager import FontProperties
# 设置中文字体
font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=12)  # 替换为你的中文字体文件路径

import sys
sys.path.append(r"C:\Users\haokw\Documents\GitHub\gaolu\MPC\高炉")
import base 


In [2]:
# 读取Excel文件

excel_path = f'C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据处理\\新输入输出模式\\1h_mean.xlsx'
df_sheet_params = pd.read_excel(excel_path, sheet_name='1h_mean_all') 

# print(df_sheet_params.info())
# print(df_sheet_params.columns)



In [3]:
df_sheet_params.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377 entries, 0 to 7376
Data columns (total 38 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   主参数时间戳    7377 non-null   datetime64[ns]
 1   主参数时间戳2   7377 non-null   int64         
 2   时间戳h      7377 non-null   int64         
 3   富氧率       7377 non-null   float64       
 4   透气性指数     7377 non-null   float64       
 5   CO        7377 non-null   float64       
 6   H2        7377 non-null   float64       
 7   CO2       7377 non-null   float64       
 8   标准风速      7377 non-null   float64       
 9   富氧流量      7377 non-null   float64       
 10  冷风流量      7377 non-null   float64       
 11  鼓风动能      7377 non-null   float64       
 12  炉腹煤气量     7377 non-null   float64       
 13  炉腹煤气指数    7377 non-null   float64       
 14  理论燃烧温度    7377 non-null   float64       
 15  顶压        7377 non-null   float64       
 16  顶压2       7377 non-null   float64       
 17  顶压3       7377

In [4]:
df = df_sheet_params.copy()
df.drop(columns=['主参数时间戳','主参数时间戳2','时间戳h', '透气性指数', 'CO', 'H2', 'CO2', '标准风速', '鼓风动能','炉腹煤气量',
                '炉腹煤气指数', '理论燃烧温度', '顶压', '顶压2', '顶压3', '顶压4', '全压差',
                '实际风速','顶温东北', '顶温西南', '顶温西北', '顶温东南', '顶温下降管', '阻力系数',
                '本小时实际喷煤量', '上小时实际喷煤量','富氧率', '冷风压力2', '热风压力2'], inplace=True)
df.info()
data = df.values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377 entries, 0 to 7376
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   富氧流量    7377 non-null   float64
 1   冷风流量    7377 non-null   float64
 2   富氧压力    7377 non-null   float64
 3   冷风压力    7377 non-null   float64
 4   热风压力    7377 non-null   float64
 5   冷风温度    7377 non-null   float64
 6   热风温度    7377 non-null   float64
 7   鼓风湿度    7377 non-null   float64
 8   设定喷煤量   7377 non-null   float64
dtypes: float64(9)
memory usage: 518.8 KB


In [5]:
df.columns.values


array(['富氧流量', '冷风流量', '富氧压力', '冷风压力', '热风压力', '冷风温度', '热风温度', '鼓风湿度',
       '设定喷煤量'], dtype=object)

In [6]:
data.shape


(7377, 9)

In [7]:
import numpy as np
from sklearn.decomposition import PCA

# 示例数据
# data = np.random.rand(100, 16)  # 假设有100个样本，每个样本16个维度的数据

# 进行PCA降维
pca = PCA()
pca.fit(data)

# 计算因子载荷矩阵
factor_loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# 物理变量的名称列表
feature_names = df.columns.values # 假设已经定义了原始物理变量的名称列表

# 将因子载荷矩阵的绝对值排序，并找出与主成分最相关的前6个物理变量
top_features_indices = np.argsort(np.abs(factor_loadings), axis=0)[::-1][:7, :]
top_features = [[feature_names[idx] for idx in indices] for indices in top_features_indices]

# 输出与主成分最关联的物理变量
for i, features in enumerate(top_features):
    print(f"主成分{i+1}与以下物理变量最相关：", features)


主成分1与以下物理变量最相关： ['富氧流量', '热风温度', '冷风温度', '鼓风湿度', '设定喷煤量', '冷风流量', '富氧压力', '冷风压力', '热风压力']
主成分2与以下物理变量最相关： ['热风温度', '冷风温度', '冷风流量', '设定喷煤量', '鼓风湿度', '设定喷煤量', '冷风流量', '热风压力', '冷风压力']
主成分3与以下物理变量最相关： ['设定喷煤量', '设定喷煤量', '设定喷煤量', '冷风流量', '冷风温度', '鼓风湿度', '设定喷煤量', '冷风流量', '冷风流量']
主成分4与以下物理变量最相关： ['冷风温度', '富氧流量', '热风温度', '冷风温度', '冷风流量', '冷风温度', '鼓风湿度', '富氧压力', '鼓风湿度']
主成分5与以下物理变量最相关： ['鼓风湿度', '鼓风湿度', '鼓风湿度', '热风温度', '热风温度', '热风温度', '冷风压力', '设定喷煤量', '冷风温度']
主成分6与以下物理变量最相关： ['冷风流量', '冷风流量', '冷风压力', '富氧压力', '富氧压力', '热风压力', '热风压力', '鼓风湿度', '富氧压力']
主成分7与以下物理变量最相关： ['冷风压力', '富氧压力', '热风压力', '冷风压力', '热风压力', '冷风压力', '冷风温度', '冷风温度', '设定喷煤量']


CCA

In [8]:
excel_path = f'C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据处理\\新输入输出模式\\1h_mean.xlsx'
df_in = pd.read_excel(excel_path, sheet_name='1h_mean_all') 
excel_path = f'C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据处理\\新输入输出模式\\1h_mean.xlsx'
df_out = pd.read_excel(excel_path, sheet_name='原始输出') 


In [9]:
df_in2 = df_in.copy()
df_in2 = df_in2.iloc[0:800,:]
df_in2.head()
df_in2.tail()
df_in2.drop(columns=['主参数时间戳','主参数时间戳2','时间戳h', '透气性指数', 'CO', 'H2', 'CO2', '标准风速', '鼓风动能','炉腹煤气量',
                '炉腹煤气指数', '理论燃烧温度', '顶压', '顶压2', '顶压3', '顶压4', '全压差',
                '实际风速','顶温东北', '顶温西南', '顶温西北', '顶温东南', '顶温下降管', '阻力系数',
                '本小时实际喷煤量', '上小时实际喷煤量','富氧率', '冷风压力2', '热风压力2'
], inplace=True)
df_in2.info()

data_in = df_in2.values
columns_name = df_in2.columns.values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   富氧流量    800 non-null    float64
 1   冷风流量    800 non-null    float64
 2   富氧压力    800 non-null    float64
 3   冷风压力    800 non-null    float64
 4   热风压力    800 non-null    float64
 5   冷风温度    800 non-null    float64
 6   热风温度    800 non-null    float64
 7   鼓风湿度    800 non-null    float64
 8   设定喷煤量   800 non-null    float64
dtypes: float64(9)
memory usage: 56.4 KB


In [10]:
df_in2 = df_in.copy()
df_in2 = df_in2.iloc[0:800,:]
df_in2.head()
df_in2.tail()
df_in2.drop(columns=['主参数时间戳','主参数时间戳2','时间戳h', 'CO', 'H2', 'CO2', 
                    
                '顶压2', '顶压3', '顶压4',
                
                '本小时实际喷煤量', '上小时实际喷煤量', '冷风压力2', '热风压力2'], inplace=True)
df_in2.info()

data_in = df_in2.values
columns_name = df_in2.columns.values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   富氧率     800 non-null    float64
 1   透气性指数   800 non-null    float64
 2   标准风速    800 non-null    float64
 3   富氧流量    800 non-null    float64
 4   冷风流量    800 non-null    float64
 5   鼓风动能    800 non-null    float64
 6   炉腹煤气量   800 non-null    float64
 7   炉腹煤气指数  800 non-null    float64
 8   理论燃烧温度  800 non-null    float64
 9   顶压      800 non-null    float64
 10  富氧压力    800 non-null    float64
 11  冷风压力    800 non-null    float64
 12  全压差     800 non-null    float64
 13  热风压力    800 non-null    float64
 14  实际风速    800 non-null    float64
 15  冷风温度    800 non-null    float64
 16  热风温度    800 non-null    float64
 17  顶温东北    800 non-null    float64
 18  顶温西南    800 non-null    float64
 19  顶温西北    800 non-null    float64
 20  顶温东南    800 non-null    float64
 21  顶温下降管   800 non-null    float64
 22  阻力

In [11]:
df_out2 = df_out.copy()
df_out2 = df_out2.iloc[1:1+800,:]
df_out2.head(5)
df_out2.tail(5)
df_out2.drop(columns=['时间戳h','时间戳10m'], inplace=True)
df_out2.info()
data_out = df_out2.values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 1 to 800
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   铁水硅含量[SI]  800 non-null    float64
 1   铁水温度[MIT]  800 non-null    float64
dtypes: float64(2)
memory usage: 12.6 KB


In [12]:
data_in.shape


(800, 25)

In [13]:
# 计算每个特征的均值和标准差
means = np.mean(data_in, axis=0)
stds = np.std(data_in, axis=0)

# 对每个特征进行归一化
normalized_data_in = (data_in - means) / stds


# 计算每个特征的均值和标准差
means = np.mean(data_out, axis=0)
stds = np.std(data_out, axis=0)

# 对每个特征进行归一化
normalized_data_out = (data_out - means) / stds


In [14]:
import numpy as np
from scipy.linalg import svd

# 生成示例数据
X = normalized_data_in  # 第一组变量，100个样本，每个样本5个特征
Y = normalized_data_out  # 第二组变量，与第一组对应，每个样本5个特征
print(X.shape)
print(Y.shape)

# 计算CCA
U, _, Vt = svd(np.dot(X.T, Y))

# 提取典型变量
C1 = np.dot(X, U[:, 0])  # 第一组典型变量
C2 = np.dot(Y, Vt.T[:, 0])  # 第二组典型变量
print(Vt.T[:, 0])


(800, 25)
(800, 2)
[0.98286288 0.18433817]


In [15]:
C2.shape


(800,)

In [16]:
np.corrcoef(C1.T, C2.T)


array([[1.        , 0.59754496],
       [0.59754496, 1.        ]])

In [17]:

from sklearn.metrics import r2_score


In [19]:
import numpy as np

# 示例数据
normalized_data_in = normalized_data_in  # 第一组变量，800个样本，每个样本9个特征
recombination_data_out = C2  # 第二组变量，800个样本，每个样本1个特征

# 计算每个特征与 normalized_data_out 的皮尔逊相关系数
correlation_coeffs = np.corrcoef(normalized_data_in.T, recombination_data_out.T)[:, -1]
# 输出结果
for i, corr_coeff in enumerate(correlation_coeffs):
    print(f"{columns_name[i]} {corr_coeff} {np.fabs(corr_coeff)}")



富氧率 -0.5262973954280458 0.5262973954280458
透气性指数 0.4941670618502941 0.4941670618502941
标准风速 0.3549398800623601 0.3549398800623601
富氧流量 -0.5130530240395157 0.5130530240395157
冷风流量 0.35495710503058764 0.35495710503058764
鼓风动能 -0.5545813678557793 0.5545813678557793
炉腹煤气量 -0.28817415573239014 0.28817415573239014
炉腹煤气指数 -0.2881742084970967 0.2881742084970967
理论燃烧温度 -0.513308390778597 0.513308390778597
顶压 -0.15083083493253427 0.15083083493253427
富氧压力 -0.18759749669941944 0.18759749669941944
冷风压力 -0.3116177582442027 0.3116177582442027
全压差 -0.28744432405020665 0.28744432405020665
热风压力 -0.28237888386983484 0.28237888386983484
实际风速 -0.6331848130133858 0.6331848130133858
冷风温度 0.07441940063757221 0.07441940063757221
热风温度 -0.6573231290167423 0.6573231290167423
顶温东北 0.44742864293968115 0.44742864293968115
顶温西南 0.40799482765655437 0.40799482765655437
顶温西北 0.4826995575120647 0.4826995575120647
顶温东南 0.48023474959409623 0.48023474959409623
顶温下降管 0.5597132654001765 0.5597132654001765
阻力系数 -0.000603851111

IndexError: index 25 is out of bounds for axis 0 with size 25